In [3]:
# Searching all file under given path
import os
def get_filepath_list(root_dir):
    file_list = []
    folder_list = []
    
    for root,dirs,files in os.walk(root_dir):
        for i_file in files:
            file_list.append(f'{root}/{i_file}')
            
        for i_dir in dirs:
            folder_list.append(f'{root}/{i_dir}')
    return file_list, folder_list

# Classify file type
def analyse_file_type(f_list):
    file_type_list = {}
    for file in f_list:
        file_type = file.split('.')[-1]
        if file_type in file_type_list.keys():
            file_type_list[file_type].append(file)
        else:
            file_type_list[file_type] = [file]
            
    return file_type_list

dir = '/home/guoqizhi/deepseek-chat/'
file_list, folder_list = get_filepath_list(dir)
file_type_list = analyse_file_type(file_list)

file_type_list['json']

['/home/guoqizhi/deepseek-chat//190-MINECRAFTFerris_Wheel_HeII_DAY_1.json',
 '/home/guoqizhi/deepseek-chat//142-DEVIL_MAY_CRY_5_dande.json',
 '/home/guoqizhi/deepseek-chat//330-MINECRAFTwow_what_this_game.json',
 '/home/guoqizhi/deepseek-chat//208-MINECRAFT_CONDUIT_HOUSE_BUILD_gao.json',
 '/home/guoqizhi/deepseek-chat//83-RING_FIT_ADVENTURE_Level_30_MAX_SWEAT.json',
 '/home/guoqizhi/deepseek-chat//400-BALDIS_BASICSMATH_IS_SCARY.json',
 '/home/guoqizhi/deepseek-chat//5-BIG_NEWS_SHARK_SHARK_SHARK_GAWRGURA_hololiveEnglish_holoMyth.json',
 '/home/guoqizhi/deepseek-chat//143-SUPER_METROID_Crocomire.json',
 '/home/guoqizhi/deepseek-chat//205-MINECRAFT_Gawr_Gura_Demolition_Derby_Inc.json',
 '/home/guoqizhi/deepseek-chat//282-_.json',
 '/home/guoqizhi/deepseek-chat//230-MINECRAFT_minecraft_build_or_else.json',
 '/home/guoqizhi/deepseek-chat//340-MINECRAFTGORBIUS.json',
 '/home/guoqizhi/deepseek-chat//258-GTA_V_shark_girl_does_CRIME.json',
 '/home/guoqizhi/deepseek-chat//139-SUPER_METROID_Comfy

In [4]:
# Merge different data
from transformers import AutoTokenizer
embedding_tokenizer = AutoTokenizer.from_pretrained('/data/aicuserData/guoqizhi/embedding_model/bge-m3')

import json
qa_dataset = []
passage_dataset = []
i = 0
for file in file_type_list['json']:
    with open(file) as f:
        data = json.load(f)
        for d in data:
            d['file'] = file.split('/')[-1]
            d['block_id'] = i
            i += 1
            # Filte extrame long query and answer
            for j in range(len(d['qa_pairs'])):
                encoding_result = embedding_tokenizer(d['qa_pairs'][j]['Gura'], add_special_tokens=False)['input_ids']
                if len(encoding_result) > 256:
                    d['qa_pairs'][j]['Gura'] = embedding_tokenizer.decode(encoding_result[:512])
            qa_dataset += d['qa_pairs']
            passage_dataset.append(d['script'])

/home/guoqizhi/anaconda3/envs/fintune-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Load embedding model
from FlagEmbedding import BGEM3FlagModel

embedding_model = BGEM3FlagModel('/data/aicuserData/guoqizhi/embedding_model/bge-m3',  use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

# Extract embedding result
import numpy
Query = [d['User'] for d in qa_dataset]
Answer = [d['Gura'] for d in qa_dataset]

Query_embedding = []
Answer_embedding = []

batch_size = 512
for i in range(len(Query)//batch_size + 1):
    Query_sentence = Query[i*batch_size: i*batch_size + batch_size]
    Answer_sentence = Query[i*batch_size: i*batch_size + batch_size]
    Query_embedding.append(embedding_model.encode(Query_sentence, batch_size=batch_size,  max_length=512)['dense_vecs'])
    Answer_embedding.append(embedding_model.encode(Answer_sentence, batch_size=batch_size,  max_length=512)['dense_vecs'])
    
Query_embedding = numpy.concatenate(Query_embedding)
Answer_embedding = numpy.concatenate(Answer_embedding)

numpy.save('query_embedding.npy', Query_embedding)
numpy.save('answer_embedding.npy', Answer_embedding)

Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 13.91it/s]


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 12.31it/s]


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 13.38it/s]


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 14.48it/s]
